In [1]:
from pathlib import Path
import requests
from tqdm import tqdm
import matplotlib.pyplot as plt

%matplotlib inline
import pandas as pd
import numpy as np
from IPython.display import display


phoible_data_url = "https://raw.githubusercontent.com/phoible/dev/v2.0/data/phoible.csv"
phobile_file_path = "./downloads/phoible.csv"

VALIDATE_RESULTS = True

if not Path(phobile_file_path).exists():
    response = requests.get(phoible_data_url, stream=True)

    with open(phobile_file_path, "wb") as fh:
        for data in tqdm(response.iter_content()):
            fh.write(data)

In [2]:
phoible_df = pd.read_csv(phobile_file_path, encoding="utf-8", engine="python")

In [13]:
phonemes_df = phoible_df[
    [
        "Phoneme",
        "Marginal",
        "SegmentClass",
        "stress",
        "syllabic",
        "short",
        "long",
        "consonantal",
        "sonorant",
        "continuant",
        "delayedRelease",
        "approximant",
        "tap",
        "trill",
        "nasal",
        "lateral",
        "labial",
        "round",
        "labiodental",
        "coronal",
        "anterior",
        "distributed",
        "strident",
        "dorsal",
        "high",
        "low",
        "front",
        "back",
        "tense",
        "retractedTongueRoot",
        "advancedTongueRoot",
        "periodicGlottalSource",
        "epilaryngealSource",
        "spreadGlottis",
        "constrictedGlottis",
        "fortis",
        "raisedLarynxEjective",
        "loweredLarynxImplosive",
        "click",
    ]
].drop_duplicates()
phonemes_df

,Phoneme,Marginal,SegmentClass,stress,syllabic,short,long,consonantal,sonorant,continuant,...,retractedTongueRoot,advancedTongueRoot,periodicGlottalSource,epilaryngealSource,spreadGlottis,constrictedGlottis,fortis,raisedLarynxEjective,loweredLarynxImplosive,click
0,a,NaN,vowel,-,+,-,-,-,+,+,...,-,-,+,-,-,-,0,-,-,0
1,aː,NaN,vowel,-,+,-,+,-,+,+,...,-,-,+,-,-,-,0,-,-,0
2,æ,NaN,vowel,-,+,-,-,-,+,+,...,-,-,+,-,-,-,0,-,-,0
3,æː,NaN,vowel,-,+,-,+,-,+,+,...,-,-,+,-,-,-,0,-,-,0
4,e,NaN,vowel,-,+,-,-,-,+,+,...,-,-,+,-,-,-,0,-,-,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97805,ɳʷ,False,consonant,-,-,-,-,+,+,-,...,0,0,+,-,-,-,-,-,-,-
97807,ȵʷ,False,consonant,-,-,-,-,+,+,-,...,0,0,+,-,-,-,-,-,-,-
97821,ʈʷ,False,consonant,-,-,-,-,+,-,-,...,0,0,-,-,-,-,-,-,-,-
97823,ȶʷ,False,consonant,-,-,-,-,+,-,-,...,0,0,-,-,-,-,-,-,-,-


In [16]:
phonemes_df[(phonemes_df.long == "+") & ~phonemes_df.Phoneme.str.endswith("ː")]

,Phoneme,Marginal,SegmentClass,stress,syllabic,short,long,consonantal,sonorant,continuant,...,retractedTongueRoot,advancedTongueRoot,periodicGlottalSource,epilaryngealSource,spreadGlottis,constrictedGlottis,fortis,raisedLarynxEjective,loweredLarynxImplosive,click
2131,tːs,NaN,consonant,-,-,-,+,+,-,-,...,0,0,-,-,-,-,-,-,-,-
3111,t̠ːʃ,NaN,consonant,-,-,-,+,+,-,-,...,0,0,-,-,-,-,-,-,-,-
5962,ŋˑ,NaN,consonant,-,-,+,+,+,+,-,...,0,0,+,-,-,-,-,-,-,-
39229,tːs,False,consonant,-,-,-,+,+,-,-,...,0,0,-,-,-,-,-,-,-,-
45615,a̤ˑ,False,vowel,-,+,+,+,-,+,+,...,-,-,+,-,+,-,0,-,-,0
45616,a̰ˑ,False,vowel,-,+,+,+,-,+,+,...,-,-,+,-,-,+,0,-,-,0
45625,e̤ˑ,False,vowel,-,+,+,+,-,+,+,...,-,-,+,-,+,-,0,-,-,0
45626,ḛˑ,False,vowel,-,+,+,+,-,+,+,...,-,-,+,-,-,+,0,-,-,0
45629,ɛ̤ˑ,False,vowel,-,+,+,+,-,+,+,...,-,-,+,-,+,-,0,-,-,0
45630,ɛ̰ˑ,False,vowel,-,+,+,+,-,+,+,...,-,-,+,-,-,+,0,-,-,0
